In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
import pickle as pk
from torch.utils.data import DataLoader, Dataset 
import time

import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
import re
import string
import codecs

def make_word_list(sentence):
    
    for c in string.punctuation:
        corpus_text = sentence.replace(c, "") 
    
    text = re.sub(r'\S*\d\S*', '', corpus_text)
    text = re.sub(r'[^\w\s]', '', text) 
    
    text = text.split() 
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [3]:
f = open("./topicclass/topicclass_train.txt", "r")

import numpy as np

sentences= []
labels = []

word_set = set()

i=0
for x in f:

  word_np_list = np.array(make_word_list(x.split("|||")[1]).split())
  sentences.append(word_np_list)

  for word in word_np_list:
    word_set.add(word)

  labels.append(x.split("|||")[0])

f.close()

In [4]:
f = open("./topicclass/topicclass_valid.txt", "r")

sentences_valid= []
labels_valid = []

word_set_valid = set()

i=0
for x in f:

  word_np_list = np.array(make_word_list(x.split("|||")[1]).split())
  sentences_valid.append(word_np_list)

  for word in word_np_list:
    word_set_valid.add(word)

  labels_valid.append(x.split("|||")[0])

f.close()

In [5]:
f = open("./topicclass/topicclass_test.txt", "r")

sentences_test= []
labels_test = []

word_set_test = set()

i=0
for x in f:

  word_np_list = np.array(make_word_list(x.split("|||")[1]).split())
  sentences_test.append(word_np_list)

  for word in word_np_list:
    word_set_test.add(word)

  labels_test.append(x.split("|||")[0])

f.close()

In [6]:
sentences_np = np.array(sentences)
labels_np = np.array(labels)
sentences_valid_np = np.array(sentences_valid)
labels_valid_np = np.array(labels_valid)
sentences_test_np = np.array(sentences_test)
labels_test_np = np.array(labels_test)

In [7]:
label_dictionary = {}

label_set = set(labels)

i=0
for lbl in label_set:
    label_dictionary[lbl] = i
    i = i+1

In [9]:
label_dictionary = {
 'Media and drama ':0,
 'Mathematics ':1,
 'Natural sciences ':2,
 'Agriculture, food and drink ':3,
 'Miscellaneous ':4,
 'Language and literature ':5,
 'Video games ':6,
 'Social sciences and society ':7,
 'History ':8,
 'Sports and recreation ':9,
 'Geography and places ':10,
 'Warfare ':11,
 'Engineering and technology ':12,
 'Art and architecture ':13,
 'Music ':14,
 'Philosophy and religion ':15
 }

In [10]:
f = open("model2.txt", "r")

i=0
dictionary={}
for x in f:
    i=i+1
    y = x.split()
    dictionary[y[0].split('_')[0]] = y[1:]
    if i > 600000:
        break
        
f.close()

In [11]:
train_dictionary={}
valid_dictionary={}
test_dictionary={}

for word in word_set:
    if word in dictionary:
        train_dictionary[word] = dictionary[word]

for word in word_set_valid:
    if word in dictionary:
        valid_dictionary[word] = dictionary[word]
        
for word in word_set_test:
    if word in dictionary:
        test_dictionary[word] = dictionary[word]
        
        
dictionary.clear()
word_set.clear()
word_set_valid.clear()
word_set_test.clear()

In [12]:
class Sentences_Dataset(Dataset):
  def __init__(self, sentences, label_dictionary, labels=None, train=True):
    self.sentences = sentences
    self.label_dictionary = label_dictionary
    self.train = train
    if(labels is not None):
      self.labels = labels
    
  def __len__(self):
    return self.sentences.shape[0]

  def __getitem__(self, index):
        
    current_sentence = self.sentences[index]
    current_label = self.label_dictionary[self.labels[index]]

    sentence_feature = []

    for word in current_sentence:
        if word in train_dictionary:
            #print(word)
            sentence_feature.append(np.array(train_dictionary[word], dtype='float'))

        
    sentence_feature_np = np.array(sentence_feature)
    sentence_feature_np.resize((70,300), refcheck=False)
    
        
        
    if(self.train):
      return torch.tensor(sentence_feature_np.astype(np.float32)), torch.tensor(current_label)
    else:
      return torch.tensor(sentence_feature_np.astype(np.float32))


class Sentences_Dataset_v(Dataset):
  def __init__(self, sentences, label_dictionary, labels=None, train=True):
    self.sentences = sentences
    self.label_dictionary = label_dictionary
    self.train = train
    if(labels is not None):
      self.labels = labels
    
  def __len__(self):
    return self.sentences.shape[0]

  def __getitem__(self, index):
        
    current_sentence = self.sentences[index]
    current_label = self.label_dictionary[self.labels[index]]

    sentence_feature = []

    for word in current_sentence:
        if word in valid_dictionary:
            #print(word)
            sentence_feature.append(np.array(valid_dictionary[word], dtype='float'))

        
    sentence_feature_np = np.array(sentence_feature)
    sentence_feature_np.resize((70,300), refcheck=False)
   
    if(self.train):
      return torch.tensor(sentence_feature_np.astype(np.float32)), torch.tensor(current_label)
    else:
      return torch.tensor(sentence_feature_np.astype(np.float32))



class Sentences_Dataset_test(Dataset):
  def __init__(self, sentences, label_dictionary, labels=None, train=True):
    self.sentences = sentences
    self.label_dictionary = label_dictionary
    self.train = train
    if(labels is not None):
      self.labels = labels
    
  def __len__(self):
    return self.sentences.shape[0]

  def __getitem__(self, index):
        
    current_sentence = self.sentences[index]

    sentence_feature = []

    for word in current_sentence:
        if word in valid_dictionary:
            #print(word)
            sentence_feature.append(np.array(test_dictionary[word], dtype='float'))

        
    sentence_feature_np = np.array(sentence_feature)
    sentence_feature_np.resize((70,300), refcheck=False)
   

    return torch.tensor(sentence_feature_np.astype(np.float32))





In [ ]:
Sentences_train_Dataset = Sentences_Dataset(sentences_np, label_dictionary, labels_np)
train_loader = DataLoader(Sentences_train_Dataset, batch_size=64, shuffle=True)

In [ ]:
t_loader_len = len(train_loader)

In [13]:
Sentences_valid_Dataset = Sentences_Dataset_v(sentences_valid_np, label_dictionary, labels_valid_np)
valid_loader = DataLoader(Sentences_valid_Dataset, batch_size=64, shuffle=False)

In [14]:
v_loader_len = len(valid_loader)

In [15]:
class ConvBlock(nn.Module):

    def __init__(self, input_channel_size, output_channel_size,kernel_size=1, stride=1, groups=1, padding=0):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(input_channel_size, 
                               output_channel_size, kernel_size=kernel_size, stride=stride, 
                               padding=padding, bias=False, groups=groups)
        self.bn1 = nn.BatchNorm2d(output_channel_size)
        self.tan = nn.Tanh()
        
    def forward(self, x):
        out =   self.tan(self.bn1(self.conv1(x)))
        return out
        

class BottleneckResidual(nn.Module):
    
    def __init__(self, input_channel_size, output_channel_size, stride=1, expand_factor=1, iteration = 0):
         super(BottleneckResidual, self).__init__()
         self.expand_factor = expand_factor
         self.stride = stride
         in_channel_expand = input_channel_size*self.expand_factor
        
         self.BottleNeck_Layers = []
         if (self.expand_factor == 1):
            self.BottleNeck_Layers.append(ConvBlock(input_channel_size, input_channel_size,
                                   kernel_size=3, stride=1, groups= input_channel_size, padding=1))
         else:     
            self.BottleNeck_Layers.append(ConvBlock(input_channel_size, in_channel_expand,kernel_size=1, stride=1))
            
            if iteration > 0:
                stride = 1
            
            self.BottleNeck_Layers.append(ConvBlock(in_channel_expand, in_channel_expand,
                                   kernel_size=3, stride=stride, groups = in_channel_expand, padding=1))
         
         self.BottleNeck_Layers.append(nn.Conv2d(in_channel_expand, output_channel_size,kernel_size=1, stride=1, bias=False))
         self.BottleNeck_Layers.append(nn.BatchNorm2d(output_channel_size))
        
         self.BottleNeck_Layers = nn.Sequential(*self.BottleNeck_Layers)
            
         if stride == 1:
            self.shortcut = nn.Conv2d(input_channel_size, output_channel_size, kernel_size=1, stride=stride, bias=False)
        
    def forward(self, x):
         out=self.BottleNeck_Layers(x)
            
         if (self.stride == 1):
            out+=self.shortcut(x)
        
         return out

In [16]:
class SenClasNet(nn.Module):
    def __init__(self, input_channels, output_channels, expansion_factor, iterations, strides, classes=5):
        super(SenClasNet, self).__init__()
        self.classes = classes
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.expansion_factor = expansion_factor
        self.iterations = iterations
        self.strides = strides
        
        self.layers = []
        for i in range(len(self.input_channels)):
            for j in range(self.iterations[i]):
                if i == 0:
                    #input_channel_size, output_channel_size,kernel_size=1, stride=1, groups=1, padding=0
                    self.layers.append(ConvBlock(input_channel_size = self.input_channels[i],
                                            output_channel_size = self.output_channels[i], 
                                            kernel_size = 3, 
                                            stride = self.strides[i], 
                                            padding=1 )
                                 )
                    
                    continue
                    
                if i == (len(input_channels) - 1):
                    self.layers.append(ConvBlock(input_channel_size = self.input_channels[i],
                                            output_channel_size = self.output_channels[i], 
                                            kernel_size = 1, 
                                            stride = 1)
                                 )                                     
                    continue
                
                #input_channel_size, output_channel_size, stride=1, expand_factor=1
                
                if j == 0:
                    ic = self.input_channels[i]
                    oc = self.output_channels[i]
                else:
                    ic = self.output_channels[i]
                    oc = self.output_channels[i]
                
                self.layers.append(BottleneckResidual(input_channel_size = ic,
                                                 output_channel_size = oc, 
                                                 stride = self.strides[i],
                                                 expand_factor = self.expansion_factor[i], 
                                                 iteration = j)
                             )
                
                
        self.layers = nn.Sequential(*self.layers)
        self.dpo1 = nn.Dropout(p=0.5)
        self.linear_label = nn.Linear(self.output_channels[-1], self.classes, bias=False)
        

    
    def forward(self, x, train=True):
        output = x
        output = self.layers(output)
        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        if train==True:
            output = self.dpo1(output)
        
        label_output = self.linear_label(output)

        return label_output

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [17]:

input_channels = [1, 32, 16, 24, 32, 64, 96, 160, 320]
output_channels = [32, 16, 24, 32, 64, 96, 160, 320, 1280]
expansion_factor = [1, 1, 6, 6, 6, 6, 6, 6, 1]
iterations = [1, 1, 2, 3, 4, 3, 3, 1, 1]
strides = [1, 1, 2, 2, 2, 1, 2, 1, 1]
'''
input_channels = [1, 32, 16, 24, 32, 64, 96]
output_channels = [32, 16, 24, 32, 64, 96, 160]
expansion_factor = [1, 1, 3, 3, 3, 3, 1]
iterations = [1, 1, 2, 3, 2, 3, 1]
strides = [1, 1, 2, 2, 2, 1, 1]
'''

classes = 16
network = SenClasNet(input_channels, output_channels, expansion_factor, iterations, strides, classes)


In [18]:
network = network.to(device)

In [ ]:
def train(model, data_loader, start_epoch, end_epoch):
    model.train()

    
    for epoch in range(start_epoch,end_epoch ):
        print('Epoch : '+str(epoch))
        avg_loss = 0.0
        avg_loss_train=0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            
            feats = feats.unsqueeze(-1).transpose(1,3).transpose(2,3)
            
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats, train=True)

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()
            avg_loss_train += loss.item()
            
            if batch_num % 250 == 0:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num, avg_loss/250))
                avg_loss = 0.0
            
            torch.cuda.empty_cache()
            
            del feats
            del labels
            del loss
           
        avg_valid_loss = 0.0
        pred_labs = []
        accuracy=0.0
        total=0.0
        
        for batch_num, (feats, labels) in enumerate(valid_loader):
            feats = feats.unsqueeze(-1).transpose(1,3).transpose(2,3)
            
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats, train=False)

            loss = criterion(outputs, labels.long())
            avg_valid_loss += loss.item()
            
            ##accuracy
            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            pred_labels = pred_labels.view(-1)
            pred_labs.append(pred_labels)
            
            accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
            total += len(labels)
            
            
        avg_valid_loss = avg_valid_loss/v_loader_len
        
        avg_loss_train = avg_loss_train/t_loader_len
        acc_percent = accuracy/total
        
        print('Epoch: {}\t Avg-Train-Loss: {:.4f}'.format(epoch, avg_loss_train))
        print('Epoch: {}\t Avg-Valid-Loss: {:.4f}'.format(epoch, avg_valid_loss))
        print('Epoch: {}\t Valid_Accuracy: {:.4f}'.format(epoch, acc_percent))
        
            
        if epoch % 2 == 0 :
            torch.save({
                        'epoch': epoch,
                        'model_state_dict':network.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': avg_loss,
                        }, "model_params_new" + str(epoch) + ".tar")

In [ ]:
torch.cuda.empty_cache()
network.apply(init_weights)

In [19]:
learningRate = 3e-3
weightDecay = 5e-5

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
#optimizer = torch.optim.Adam(network.parameters(), learningRate)


In [ ]:
train(network, train_loader, 0, 3)

In [ ]:
load_weights = torch.load("model_params_new2.tar")
network.load_state_dict(load_weights['model_state_dict'])
optimizer.load_state_dict(load_weights['optimizer_state_dict'])
epoch = load_weights['epoch']
loss= load_weights['loss']

lr = learningRate * 0.1

for param_group in optimizer.param_groups:
    param_group['lr'] = lr

In [ ]:
train(network, train_loader, 3, 5)

In [ ]:
load_weights = torch.load("model_params_new4.tar")
network.load_state_dict(load_weights['model_state_dict'])
optimizer.load_state_dict(load_weights['optimizer_state_dict'])
epoch = load_weights['epoch']
loss= load_weights['loss']

lr = learningRate * 0.01

for param_group in optimizer.param_groups:
    param_group['lr'] = lr

In [ ]:
train(network, train_loader, 5, 7)

In [ ]:
load_weights = torch.load("model_params_new6.tar")
network.load_state_dict(load_weights['model_state_dict'])
optimizer.load_state_dict(load_weights['optimizer_state_dict'])
epoch = load_weights['epoch']
loss= load_weights['loss']

lr = learningRate * 0.001

for param_group in optimizer.param_groups:
    param_group['lr'] = lr

In [ ]:
train(network, train_loader, 7, 9)

In [ ]:
load_weights = torch.load("model_params_new12.tar")
network.load_state_dict(load_weights['model_state_dict'])
optimizer.load_state_dict(load_weights['optimizer_state_dict'])
epoch = load_weights['epoch']
loss= load_weights['loss']

lr = learningRate * 0.0001

for param_group in optimizer.param_groups:
    param_group['lr'] = lr

In [ ]:
train(network, train_loader, 13, 15)

TESTING

In [ ]:
Sentences_valid_Dataset = Sentences_Dataset_v(sentences_valid_np, label_dictionary, labels_valid_np)
valid_loader = DataLoader(Sentences_valid_Dataset, batch_size=64, shuffle=False)

Sentences_test_Dataset = Sentences_Dataset_test(sentences_test_np, label_dictionary)
test_loader = DataLoader(Sentences_test_Dataset, batch_size=1, shuffle=False)

In [20]:
class_list = ['Media and drama',
 'Mathematics',
 'Natural sciences',
 'Agriculture, food and drink',
 'Miscellaneous',
 'Language and literature',
 'Video games',
 'Social sciences and society',
 'History',
 'Sports and recreation',
 'Geography and places',
 'Warfare',
 'Engineering and technology',
 'Art and architecture',
 'Music',
 'Philosophy and religion']

In [21]:
load_weights = torch.load("model_params_new4.tar")
network.load_state_dict(load_weights['model_state_dict'])
optimizer.load_state_dict(load_weights['optimizer_state_dict'])
epoch = load_weights['epoch']
loss= load_weights['loss']


def test(model, data_loader):
    accuracy=0
    total=0
    for batch_num, (feats, labels) in enumerate(data_loader):
        feats = feats.unsqueeze(-1).transpose(1,3).transpose(2,3)
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats, train=False)
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        for x in pred_labels:
            print(class_list[x])
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        
        total += len(labels)
        
    print(accuracy/total)

In [22]:
test(network, valid_loader)

Sports and recreation
Sports and recreation
Sports and recreation
Media and drama
History
Music
Music
Music
Social sciences and society
History
Media and drama
Warfare
Music
Media and drama
Sports and recreation
Natural sciences
Geography and places
Sports and recreation
Social sciences and society
Media and drama
Sports and recreation
Music
Warfare
Sports and recreation
Geography and places
Natural sciences
Engineering and technology
Warfare
Social sciences and society
Agriculture, food and drink
Music
Warfare
Warfare
Warfare
Video games
Music
Music
Social sciences and society
Natural sciences
Sports and recreation
Warfare
Music
Engineering and technology
Social sciences and society
Media and drama
Language and literature
Natural sciences
Media and drama
Natural sciences
Engineering and technology
Natural sciences
Art and architecture
Engineering and technology
Warfare
Warfare
Media and drama
History
Geography and places
Warfare
Sports and recreation
Social sciences and society
Media 

Media and drama
Natural sciences
Social sciences and society
Natural sciences
Natural sciences
Media and drama
Music
Media and drama
Sports and recreation
Sports and recreation
Media and drama
Geography and places
Agriculture, food and drink
Geography and places
Music
History
Warfare
Media and drama
Media and drama
Warfare
Sports and recreation
History
Social sciences and society
Sports and recreation
Warfare
Geography and places
Art and architecture
Warfare
Geography and places
Social sciences and society
Sports and recreation
Music
Geography and places
Warfare
Engineering and technology
Engineering and technology
Geography and places
Music
Social sciences and society
Geography and places
Music
Natural sciences
Warfare
Art and architecture
Media and drama
History
Music
Engineering and technology
Warfare
Warfare
Music
Music
Media and drama
Warfare
Video games
Sports and recreation
Media and drama
Geography and places
Warfare
Sports and recreation
Media and drama
Engineering and technol

In [23]:
Sentences_test_Dataset = Sentences_Dataset_test(sentences_test_np, label_dictionary)
test_loader = DataLoader(Sentences_test_Dataset, batch_size=64, shuffle=False)

In [ ]:
torch.cuda.empty_cache()

In [24]:
def test(model, data_loader):
    accuracy=0
    total=0
    for batch_num, feats in enumerate(data_loader):
        feats = feats.unsqueeze(-1).transpose(1,3).transpose(2,3)
        feats = feats.to(device)
        outputs = model(feats, train=False)
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        for x in pred_labels:
            print(class_list[x])

In [25]:
test(network, test_loader)

Engineering and technology
Media and drama
Sports and recreation
Sports and recreation
History
Geography and places
Music
Sports and recreation
Warfare
Natural sciences
History
Media and drama
History
Art and architecture
Media and drama
Natural sciences
Engineering and technology
Art and architecture
Geography and places
Geography and places
Video games
Art and architecture
Media and drama
Sports and recreation
Engineering and technology
Social sciences and society
Warfare
Engineering and technology
Social sciences and society
Natural sciences
Warfare
Social sciences and society
Media and drama
Sports and recreation
Video games
Geography and places
Sports and recreation
Geography and places
Warfare
Media and drama
Sports and recreation
Art and architecture
Sports and recreation
Sports and recreation
Natural sciences
Natural sciences
Geography and places
Media and drama
Sports and recreation
Philosophy and religion
Video games
Media and drama
History
Warfare
Natural sciences
Natural sc

Warfare
Art and architecture
Art and architecture
History
Social sciences and society
Music
Music
Media and drama
Sports and recreation
Music
Sports and recreation
History
Media and drama
Warfare
Media and drama
Sports and recreation
Sports and recreation
Natural sciences
Geography and places
Warfare
Sports and recreation
Sports and recreation
Natural sciences
Media and drama
Warfare
Sports and recreation
Geography and places
Sports and recreation
Art and architecture
Media and drama
Engineering and technology
Music
Engineering and technology
Music
Engineering and technology
Natural sciences
Geography and places
Natural sciences
Natural sciences
Social sciences and society
Warfare
Media and drama
Sports and recreation
Media and drama
Sports and recreation
Engineering and technology
History
Natural sciences
Media and drama
Music
Warfare
Warfare
Sports and recreation
Media and drama
Social sciences and society
Sports and recreation
Geography and places
Engineering and technology
Sports a